The first several cells are used for some common use like loading data.

Link for kaggle competition:
https://www.kaggle.com/competitions/ml-olympiad-predicting-earthquake-damage/data?select=train.csv

If you have any idea, you can type it on your cells.

In case Your code is overwritten, you can also write the code locally then copy and paste on there. This one is mainly for sharing and combining together.

In [ ]:
import pandas as pd
import pytorch as py
import numpy as np
import torch
import torch.nn as nn

train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Next: cell

*   List item
*   List item



# Jinhao Luo parts
The cells below are for Jinhao Luo:
some kind of neural networks

In [ ]:
from matplotlib import scale
from sklearn.discriminant_analysis import StandardScaler
from sklearn.utils import compute_class_weight
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, recall_score

import pandas as pd
import numpy as np

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load data
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
test_building_id = test_df["building_id"].copy()


# deal with categorical features
cat_cols = train_df.select_dtypes(include=["object"]).columns.tolist()
for col in cat_cols:
    train_df[col] = train_df[col].astype("category").cat.codes
    test_df[col] = test_df[col].astype("category").cat.codes

# prepare for features and labels
X = train_df.drop(["building_id", "damage_grade"], axis=1).values
Y = train_df["damage_grade"].values.astype(int) - 1

X_test = test_df.drop(["building_id"], axis=1).values

# Normalize for data -- (ljh key change point)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# compute for weights for classes -- to deal with the problem of class's unbalance
class_weights = compute_class_weight("balanced", classes=np.unique(Y), y=Y)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

"""
# define our model
class MyNet(nn.Module):
    def __init__(self, input_dim):
        super(MyNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2),  # continue to lower dropout bility
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(
                16, 3
            ),  # output layer will be 3 , because damage_grade is a 3-class classification task
        )

    def forward(self, x):
        return self.layers(x)
"""


# """ My net before
class MyNet(nn.Module):
    def __init__(self, input_dim):
        super(MyNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.4),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.4),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(
                16, 3
            ),  # output layer will be 3 , because damage_grade is a 3-class classification task
        )

    def forward(self, x):
        return self.layers(x)


# """


# cross validation
fold_num = 5
epoch_num = 100
kfold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X, Y)):
    X_train, X_val = X[train_idx], X[val_idx]
    Y_train, Y_val = Y[train_idx], Y[val_idx]

    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(Y_train, dtype=torch.long),
    )
    val_dataset = TensorDataset(
        torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.long)
    )
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    # initialize model, criterion and optimizer
    model = MyNet(X_train.shape[1]).to(device)
    # criterion = nn.CrossEntropyLoss()
    criterion = nn.CrossEntropyLoss(weight=class_weights)  # Based on weighted class
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    best_model = None
    best_val_loss = float("inf")

    # parameters for early stopping
    patience = 40  # when validation loss doesn't improve for 10 epochs, stop training
    no_improve_coutners = 0  # counter
    delta = 0.001  # minimum change in validation loss to qualify as an improvement
    early_stop = False  # sign for early stopping

    # training loop
    for epoch in range(epoch_num):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

        # validation
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []

        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss = criterion(pred, yb)
                val_loss += loss.item()

                # get predicted features
                _, predicted = torch.max(pred, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_targets.extend(yb.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = accuracy_score(val_targets, val_preds)
        val_recall = recall_score(val_targets, val_preds, average="macro")
        val_f1 = f1_score(val_targets, val_preds, average="macro")

        if val_loss < best_val_loss - delta:  # validation loss have enough improvement
            best_val_loss = val_loss
            best_model = model.state_dict()
            # save best model
            torch.save(best_model, f"best_model_fold_{fold}.pth")
            no_improve_coutners = 0  # reset counter
        else:
            no_improve_coutners += 1  # increment counter

            if no_improve_coutners >= patience:  # no improvement for 'patience' epochs
                early_stop = True
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

        print(
            f"Fold {fold+1}, Epoch {epoch+1}, Val Loss: {val_loss:.4f}, "
            f"Val Accuracy: {val_acc:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}"
        )

    accuracies.append(val_acc)

mean_acc = np.mean(accuracies)
print(f"Mean Accuracy: {mean_acc:.4f}")

# Use best model
best_model = MyNet(X.shape[1]).to(device)
best_model.load_state_dict(torch.load(f"best_model_fold_{fold_num-1}.pth"))
best_model.eval()

# predict on train set
X_full_tensor = torch.tensor(X, dtype=torch.float32).to(device)
with torch.no_grad():
    y_pred_logits = best_model(X_full_tensor)
    _, y_pred_full = torch.max(y_pred_logits, 1)

y_pred_full = y_pred_full.cpu().numpy()

# give a measurement on our NN
print("Accuracy on full training set:", accuracy_score(Y, y_pred_full))
print("\nClassification Report:\n", classification_report(Y, y_pred_full))
recall = recall_score(Y, y_pred_full, average="macro")
f1 = f1_score(Y, y_pred_full, average="macro")
print(f"\nMacro Recall on full training set: {recall:.4f}")
print(f"Macro F1-score on full training set: {f1:.4f}")

# predict on test set
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
with torch.no_grad():
    y_pred_logits = best_model(X_test_tensor)
    _, y_pred_test = torch.max(y_pred_logits, 1)

y_pred_test = y_pred_test.cpu().numpy()


# Save submission
y_pred_test = y_pred_test + 1
submission = pd.DataFrame(
    {"building_id": test_building_id, "damage_grade": y_pred_test}
)
submission.to_csv("data/submission.csv", index=False)
print("\nYes Submission saved to 'data/submission.csv'")


## Before the improvement

network

```python
class MyNet(nn.Module):
    def __init__(self, input_dim):
        super(MyNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.4),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.4),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(
                16, 3
            ),  # output layer will be 3 , because damage_grade is a 3-class classification task
        )

    def forward(self, x):
        return self.layers(x)

```



outcome

```
Mean Accuracy: 0.5382
Accuracy on full training set: 0.5465
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.55      0.60       729
           1       0.53      0.91      0.67      1968
           2       0.00      0.00      0.00      1303

    accuracy                           0.55      4000
   macro avg       0.39      0.49      0.42      4000
weighted avg       0.38      0.55      0.44      4000
```





## After the 1st improvement



I add the

1. \# compute for weights for classes -- to deal with the problem of class's unbalance

   ```python
   class_weights = compute_class_weight("balanced", classes=np.unique(Y), y=Y)
   class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
   ```

   

2. \# Normalize for data -- (ljh key change point)

   ```python
   scaler = StandardScaler()
   
   X = scaler.fit_transform(X)
   
   X_test = scaler.transform(X_test)
   ```

   

3. Change the network ( it seems a little failed)

```python
class MyNet(nn.Module):
    def __init__(self, input_dim):
        super(MyNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2),  # continue to lower dropout bility
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(
                16, 3
            ),  # output layer will be 3 , because damage_grade is a 3-class classification task
        )

    def forward(self, x):
        return self.layers(x)

```

outcome

```
Mean Accuracy: 0.5382
Accuracy on full training set: 0.5465

Mean Accuracy: 0.4682
Accuracy on full training set: 0.476

Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.78      0.61       729
           1       0.65      0.09      0.16      1968
           2       0.44      0.88      0.59      1303

    accuracy                           0.48      4000
   macro avg       0.53      0.59      0.46      4000
weighted avg       0.55      0.48      0.39      4000
```

It's sad that the outcome become less, So I try to change the early stop longer first. (from delta 0.001 and patience 20 to (0.001,40), and also roll back to last net structure.

#### change back to before net and patience to 40

outcome

```
Mean Accuracy: 0.4813
Accuracy on full training set: 0.475

Classification Report:
               precision    recall  f1-score   support

           0       0.53      0.77      0.63       729
           1       0.61      0.10      0.17      1968
           2       0.44      0.88      0.58      1303

    accuracy                           0.47      4000
   macro avg       0.53      0.58      0.46      4000
weighted avg       0.54      0.47      0.39      4000
```

#### Cancel the normalization?

outcome

```
Mean Accuracy: 0.4788
Accuracy on full training set: 0.479

Classification Report:
               precision    recall  f1-score   support

           0       0.53      0.76      0.63       729
           1       0.59      0.12      0.20      1968
           2       0.44      0.87      0.58      1303

    accuracy                           0.48      4000
   macro avg       0.52      0.58      0.47      4000
weighted avg       0.53      0.48      0.40      4000
```

So I may need more experiment on that to get a better score.



# Denis Chen parts
The cells below are for Denis Chen

In [ ]:
import sys
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics

if(len(sys.argv) < 2):
    print("Usage: earthquake_prediction.py <training_data> [test_data | cross_validation_test_size] [write_to_csv]")
    sys.exit()

input = pd.read_csv(sys.argv[1])
X = input[["has_superstructure_adobe_mud","has_superstructure_mud_mortar_stone","has_superstructure_stone_flag","has_superstructure_cement_mortar_stone","has_superstructure_mud_mortar_brick","has_superstructure_cement_mortar_brick","has_superstructure_timber","has_superstructure_bamboo","has_superstructure_rc_non_engineered","has_superstructure_rc_engineered","has_superstructure_other","has_secondary_use","has_secondary_use_agriculture","has_secondary_use_hotel","has_secondary_use_rental","has_secondary_use_institution","has_secondary_use_school","has_secondary_use_industry","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"]].to_numpy()
y = input[["damage_grade"]].to_numpy().reshape(-1)

write_csv = False
cross_validation = -1

if(len(sys.argv) >= 4 and sys.argv[3] == "write"):
    write_csv = True

if(os.path.exists(sys.argv[2])):
    test = pd.read_csv(sys.argv[2])
else:
    cross_validation = float(sys.argv[2])

if(cross_validation == -1):
    X_train = X
    y_train = y
    X_test = test[["has_superstructure_adobe_mud","has_superstructure_mud_mortar_stone","has_superstructure_stone_flag","has_superstructure_cement_mortar_stone","has_superstructure_mud_mortar_brick","has_superstructure_cement_mortar_brick","has_superstructure_timber","has_superstructure_bamboo","has_superstructure_rc_non_engineered","has_superstructure_rc_engineered","has_superstructure_other","has_secondary_use","has_secondary_use_agriculture","has_secondary_use_hotel","has_secondary_use_rental","has_secondary_use_institution","has_secondary_use_school","has_secondary_use_industry","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"]].to_numpy()
    y_test = np.array([])
else:
    print("Cross Validation:" + str(cross_validation))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = cross_validation, random_state = 0)

print(X_train.shape)
clf = BernoulliNB()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)

#print(prediction.reshape((-1,1)).shape)
#print(test["building_id"].to_numpy().reshape((-1,1)).shape)

if(write_csv):
    print("Predictions written to csv")
    output = np.concatenate(((X_test["building_id"].to_numpy()).reshape((-1,1)), prediction.reshape((-1,1))), axis = 1).astype(int)
    np.savetxt("output.csv", output, fmt='%d', delimiter = ",", header = "building_id,damage_grade", comments='')

if(y_test.size > 0):
    print(metrics.accuracy_score(y_test, prediction))
    print(metrics.f1_score(y_test, prediction, average='macro'))

# Yicheng Ge parts
The cells below are for Yicheng Ge

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

train_data = pd.read_csv('/data/train.csv')
test_data = pd.read_csv('/data/test.csv')
test_object_columns = test_data.select_dtypes(include=['object']).columns
test_data_encoded = pd.get_dummies(test_data, columns=test_object_columns)
test_X = test_data_encoded.drop(['building_id'], axis=1)

object_columns = train_data.select_dtypes(include=['object']).columns
train_data_encoded = pd.get_dummies(train_data, columns=object_columns)
X = train_data_encoded.drop(['building_id', 'damage_grade'], axis=1)
y = train_data_encoded['damage_grade']

# 创建线性回归模型对象
#---------train on old data------------
model = LinearRegression()

# 使用训练集训练模型
model.fit(X, y)

# 在验证集上进行预测
y_pred = model.predict(test_X)
y_pred = y_pred.astype(int)

# 查看模型系数
coefficients = model.coef_

# 查看模型截距
intercept = model.intercept_

print(f"模型系数: {coefficients}")
print(f"模型截距: {intercept}")

# 创建提交结果的DataFrame
submission = pd.DataFrame({
    "building_id": test_data['building_id'],
    "damage_grade": y_pred
})

# 保存为csv文件
submission.to_csv('submission.csv', index=False)

print(f"\n✅ Submission saved to:'submission.csv'")


模型系数: [-0.01160897  0.00264301 -0.00355638  0.00576985  0.05712092  0.21299997
  0.0008602  -0.03937349  0.03187101 -0.17235406  0.02542759  0.04636667
 -0.15564023 -0.07583848 -0.13285055 -0.00377469 -0.01548817 -0.02490501
 -0.15078038 -0.10927813 -0.2989145  -0.19309835  0.04005061 -0.05471301
  0.00948364 -0.06299941 -0.02386116  0.0118964   0.00440632 -0.01630272
  0.06814449 -0.19442008  0.14848146 -0.06953992  0.04733404  0.04594458
  0.035885   -0.08182958  0.00543534  0.01347879 -0.08081396  0.00174775
  0.06015208 -0.06067713  0.03797399 -0.04195916  0.0646623  -0.05458788
  0.1168224  -0.0257731  -0.03646142  0.02278153 -0.08290071  0.12222928
 -0.25431575  0.02663659 -0.18192662 -0.18538101  0.10935821  0.37020827
  0.05331021  0.06065161 -0.11999117  0.02347214  0.03586742]
模型截距: 1.8501449654939115

✅ Submission saved to:'submission.csv'


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
#---------train after improvement----------
train_data_without_id = train_data_encoded.drop(['building_id'], axis=1)#data set without id
full_correlation_matrix = train_data_without_id.corr().round(2)
target_corr = full_correlation_matrix['damage_grade'].drop('damage_grade')
#select the data correlation bigger than 0.25
selected_features = target_corr[abs(target_corr) > 0.25].index.tolist()

# 从 X_train 中选取选定特征的数据，得到新的训练集特征数据
X_train_selected = X[selected_features]
X_test_selected = test_data_encoded[selected_features]

# 创建线性回归模型对象
model = LinearRegression()

# 使用训练集训练模型
model.fit(X_train_selected, y)

y_pred = model.predict(X_test_selected)
y_pred = y_pred.astype(int)




# 查看模型系数
coefficients = model.coef_

# 查看模型截距
intercept = model.intercept_

print(f"模型系数: {coefficients}")
print(f"模型截距: {intercept}")
submission = pd.DataFrame({
    "building_id": test_data['building_id'],
    "damage_grade": y_pred
})

# 保存为csv文件
submission.to_csv('submission_afterimprove.csv', index=False)

print(f"\n✅ Submission saved to:'submission_afterimprove.csv'")

模型系数: [ 0.00330215 -0.00612577  0.20952507 -0.21769286 -0.10686228 -0.18014633
  0.15572031 -0.13446457 -0.16318387  0.04931256 -0.03621514]
模型截距: 2.009165668232567

✅ Submission saved to:'submission_afterimprove.csv'


# Xiaocheng Ma parts
The cells below are for Xiaocheng Ma

In [ ]:
# ======================================
# 1. Improved Earthquake Damage SVM Model
# ======================================
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight

# ======================================
# 2. Load data
# ======================================
train_path = os.path.join(data_path, 'train.csv')
test_path = os.path.join(data_path, 'test.csv')

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

test_building_id = test_df["building_id"].copy()

# ======================================
# 3. Encode categorical columns
# ======================================
cat_cols = train_df.select_dtypes(include=["object"]).columns.tolist()
for col in cat_cols:
    train_df[col] = train_df[col].astype("category").cat.codes
    test_df[col] = test_df[col].astype("category").cat.codes

# ======================================
# 4. Prepare Features
# ======================================
X = train_df.drop(["building_id", "damage_grade"], axis=1).values
Y = train_df["damage_grade"].values.astype(int) - 1
X_test = test_df.drop(["building_id"], axis=1).values

# === Remove low-variance features (helps SVM) ===
selector = VarianceThreshold(threshold=0.01)
X = selector.fit_transform(X)
X_test = selector.transform(X_test)

# === Scale features ===
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# === Optional PCA (recommended if features > 50) ===
pca = PCA(n_components=0.95, random_state=42)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

# === Compute class weights ===
class_weights = compute_class_weight("balanced", classes=np.unique(Y), y=Y)
class_weights_dict = {i: w for i, w in enumerate(class_weights)}

# ======================================
# 5. Hyperparameter Search
# ======================================
param_grid = {
    "kernel": ["rbf", "poly"],
    "C": [0.5, 1, 3, 5],
    "gamma": ["scale", 0.01, 0.05, 0.1],
    "degree": [2, 3]  # for poly kernel
}

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

svm = SVC(class_weight=class_weights_dict)

grid = GridSearchCV(
    svm,
    param_grid,
    cv=kfold,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2
)

print("🔎 Running GridSearchCV for hyperparameter tuning...\n")
grid.fit(X, Y)

print("\n🔥 BEST PARAMS FOUND:", grid.best_params_)
print("🔥 BEST CV SCORE:", grid.best_score_)

# ======================================
# 6. Train final model using best parameters
# ======================================
final_model = grid.best_estimator_
final_model.fit(X, Y)

# Evaluate training performance
train_pred = final_model.predict(X)
print("\nTraining Accuracy:", accuracy_score(Y, train_pred))
print("\nClassification Report:\n", classification_report(Y, train_pred))

# ======================================
# 7. Predict test set
# ======================================
y_test_pred = final_model.predict(X_test)
y_test_pred = y_test_pred + 1  # convert back to 1/2/3

# Save submission
submission = pd.DataFrame({
    "building_id": test_building_id,
    "damage_grade": y_test_pred
})

submission.to_csv(os.path.join(data_path, "submission_svm_optimized.csv"), index=False)
print("\n✅ Submission saved as 'submission_svm_optimized.csv' in Google Drive")









Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fold 1 Accuracy: 0.4913
Fold 2 Accuracy: 0.4963
Fold 3 Accuracy: 0.4950
Fold 4 Accuracy: 0.4825
Fold 5 Accuracy: 0.4975

Mean Cross-Validation Accuracy: 0.4925

Accuracy on full training set: 0.58275

Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.83      0.70       729
           1       0.75      0.35      0.48      1968
           2       0.50      0.80      0.61      1303

    accuracy                           0.58      4000
   macro avg       0.62      0.66      0.60      4000
weighted avg       0.64      0.58      0.56      4000


✅ Submission saved to: 'submission.csv'


Last code cells: